# Pollution and mental performance in BCN

## Inicialització

In [ ]:
import warnings
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

df = pd.read_csv('CitieSHealth_BCN_DATA_PanelStudy_20220414.csv')

## Anàlisi exploratori de dades inicial

In [ ]:
print(f"Les seves dimensions són: {df.shape[0]} individus i {df.shape[1]} variables")
print(f"Llistat de variables: {list(df.columns)}")

Codifiquem algunes variables com a categòriques, i n'eliminem algunes ja que no són rellevants o contenen informació repetida en altres.
Més endavant valorarem si, estudiant les correlacions, cal treure'n més.

In [ ]:
list_to_cat = ['ID_Zenodo', 'year', 'month', 'day', 'dayoftheweek', 'hour',
               'mentalhealth_survey', 'bienestar', 'energia', 'estres', 'sueno', 
               'occurrence_stroop', 'precip_12h_binary', 'precip_24h_binary',
               'noise_total_LDEN_55']

for name in list_to_cat:
    df[name] = df[name].astype('category')

In [ ]:
covid = ['covid_work', 'covid_mood', 'covid_sleep', 'covid_espacios', 'covid_aire',
                'covid_motor', 'covid_electric', 'covid_bikewalk', 'covid_public_trans']

In [ ]:
eliminations = ['date_all', 'mean_incongruent', 'correct', 'response_duration_ms',
                'mean_congruent', 'horasfuera', 'performance', 'inhib_control', 'z_mean_incongruent',
                'z_inhib_control', 'no2bcn_12h_x30', 'no2bcn_24h_x30', 'no2gps_12h_x30',
                'no2gps_24h_x30', 'min_gps', 'hour_gps', 'sec_noise55_day', 'sec_noise65_day',
                'sec_greenblue_day', 'Houron', 'Houroff', 'start_day', 'start_month', 'start_year',
                'start_hour', 'end_day', 'end_month', 'end_year', 'end_hour', 'Totaltime', 
                'Totaltime_estimated', 'mentalhealth_survey', 'stroop_test', 'occurrence_stroop',
                'yearbirth', 'year', 'month', 'day', 'hour', 'µgm3']
df = df.drop(eliminations, axis=1)

Procedim a realitzar alguns gràfics

In [ ]:
# Gráfics de distribució per variables numèriques
num_cols = df.select_dtypes(include='number').columns
num_plots = len(num_cols)
num_rows = math.ceil(num_plots / 3)

fig, axs = plt.subplots(num_rows, 3, figsize=(15, num_rows*5))
axs = axs.ravel()  

for i in range(len(axs)):
    if i < len(num_cols):
        sns.histplot(df[num_cols[i]], kde=True, ax=axs[i])
        axs[i].set_title(f'Distribució de {num_cols[i]}')
        axs[i].set_xlabel(num_cols[i])
        axs[i].set_ylabel('Frequència')
    else:
        fig.delaxes(axs[i]) 

plt.tight_layout()
plt.show()


In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = df.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=df, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

Visualitzem els missing values per variable

In [ ]:
import missingno as msno
msno.matrix(df)

missing_values1 = df.isnull().sum()
missing_values1 = missing_values1[missing_values1 > 0]
missing_values1 = pd.DataFrame(missing_values1, columns=['missing_values'])
missing_values1

In [ ]:
df.shape

In [ ]:
# Barplot amb el numero de missing values per fila 
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x=df.isnull().sum(axis=1))
plt.title(f'Número de missing values per mostra')
plt.xlabel('Número de missing values')
plt.ylabel('Comptador')
plt.xticks(rotation=45, ha='right')  # Rotar les etiquetes de l'eix x per a una major llegibilitat
plt.show()


In [ ]:
# Creem un nou DataFrame només amb les variables numèriques
df_numeric = df.select_dtypes(include=[np.number])

# Calculem la matriu de correlació
corr = df_numeric.corr()

# Creem una màscara per a la part superior del triangle
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)

# Plotejem la matriu de correlació
plt.figure(figsize=(10, 8))
sns.heatmap(corr, mask=mask, annot=False, cmap='coolwarm')
plt.title('Matriu de correlacions')
plt.show()

In [ ]:
df_numeric = df[['no2gps_24h', 'z_performance']]
sns.pairplot(df_numeric, kind='reg', plot_kws={'line_kws':{'color':'red'}})
plt.show()

In [ ]:
pca = PCA(n_components=2)

no_miss = df_numeric.dropna()
X_r = pca.fit_transform(no_miss)

# Plot PCA results
plt.figure()

plt.scatter(X_r[:, 0], X_r[:, 1], color='navy', alpha=.8)
plt.title('PCA of numeric variables')

plt.show()

## Preprocessament

CAL SEPARAR EN TRAIN I TEST PERÒ ÉS PER VEURE MÉS O MENYS QUÈ PODEM FER

Eliminem els missing de performance

In [ ]:
deleted_na_perf = df.dropna(subset="z_performance")

missing_values1 = deleted_na_perf.isnull().sum()
missing_values1 = missing_values1[missing_values1 > 0]
missing_values1 = pd.DataFrame(missing_values1, columns=['missing_values'])
missing_values1

In [ ]:
deleted_na_perf.shape

In [ ]:
deleted_na_perf = deleted_na_perf[deleted_na_perf.isnull().sum(axis=1) < 10]

In [ ]:
deleted_na_perf.shape

In [ ]:
missing_values1 = deleted_na_perf.isnull().sum()
missing_values1 = missing_values1[missing_values1 > 0]
missing_values1 = pd.DataFrame(missing_values1, columns=['missing_values'])
missing_values1

In [ ]:
deleted_na_cols = deleted_na_perf.dropna(axis=1, thresh=len(deleted_na_perf)-280)

In [ ]:
missing_values1 = deleted_na_cols.isnull().sum()
missing_values1 = missing_values1[missing_values1 > 0]
missing_values1 = pd.DataFrame(missing_values1, columns=['missing_values'])
missing_values1

In [ ]:
msno.matrix(deleted_na_cols)

In [ ]:
unique_na = deleted_na_perf.drop_duplicates("ID_Zenodo")

In [ ]:
missing_values1 = unique_na.isnull().sum()
missing_values1 = missing_values1[missing_values1 > 0]
missing_values1 = pd.DataFrame(missing_values1, columns=['missing_values'])
missing_values1

## Partició en Train i Test

In [ ]:
from sklearn.model_selection import train_test_split
X = deleted_na_cols[deleted_na_cols.columns.drop("z_performance")]
y = deleted_na_cols["z_performance"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
num_cols = X_train.select_dtypes(include='number').columns
print(X_train[num_cols])

In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

Usem un imputador per emplenar la resta de missings

In [ ]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

num_cols = X_train.select_dtypes(include='number').columns

numeric_delna = X_train[num_cols]
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(numeric_delna)

X_train[num_cols] = imp_mean.transform(numeric_delna)
X_test[num_cols] = imp_mean.transform(X_test[num_cols])

In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

In [ ]:
print(X_train[num_cols])

Ens queden les categòriques. Emplenem amb KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

class KNNImputer:
    def __init__(self, n_neighbors = 5):
        self.n_neighbors = n_neighbors
    def fit(self, X, numeric_cols):
        self.X = X.copy() # suposem que les numeriques estan imputades
        self.numeric = numeric_cols
        self.vecinos = NearestNeighbors(n_neighbors=self.n_neighbors).fit(X.dropna()[numeric_cols])
    def _get_mode(self, variable, indexs):
        vecinos = [self.X.iloc[[i]].iloc[0] for i in indexs[0]]
        valors = [v.iloc[variable] for v in vecinos]
        unique, counts = np.unique(valors, return_counts = True)
        mode = unique[np.argmax(counts)]
        return mode
    def _impute_row(self, row):
        dist, indexs = self.vecinos.kneighbors([row[self.numeric]])
        for variable, value in enumerate(row.isna()):
            if value:
                mode = self._get_mode(variable, indexs)
                row.iloc[variable] = mode
        return row
    def transform(self, X):
        X_imputed = []
        for ind, row in X.iterrows():
            new_row = self._impute_row(row)
            X_imputed.append(new_row)
        return pd.DataFrame(X_imputed, columns=X.columns)

In [ ]:
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns

imputer = KNNImputer()

imputer.fit(X_train, num_cols)

X_train_new = imputer.transform(X_train)
X_test_new = imputer.transform(X_test)

X_train_new.head()

In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

In [ ]:
X_train_new.shape

In [ ]:
# Mode imputation
"""
for col in deleted_na_perf.columns:
    deleted_na_perf[col] = deleted_na_perf[col].fillna(value=deleted_na_perf[col].mode()[0])
"""

Tractem les variables textuals de COVID

In [ ]:
covid = ['covid_work', 'covid_mood', 'covid_sleep', 'covid_espacios', 'covid_aire',
                'covid_motor', 'covid_electric', 'covid_bikewalk', 'covid_public_trans']

for column in covid:
    X_train_new[column] = X_train_new[column].apply(lambda x: 'Yes' if 'no' in x.lower() or 'igual' in x.lower() else 'No')
    X_test_new[column] = X_test_new[column].apply(lambda x: 'Yes' if 'no' in x.lower() or 'igual' in x.lower() else 'No')

# print categories for covid variables

for col in covid:
    print(f'{col}: {X_train_new[col].unique()}')

In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train_new.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train_new, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

In [ ]:
# Creem una nova variable anomenada 'covid_afecta' binaria, on True indica que la persona ha estat afectada per la covid i False en cas contrari.
# Aquesta variable es crea a partir de les variables de la llista 'covid', i fem la moda de les respostes per a cada individu.

for row in X_train_new.iterrows():
    # Si la moda de les respostes de les variables de la llista 'covid' és 'Yes', la persona ha estat afectada per la covid
    if row[1][covid].mode()[0] == 'Yes':
        X_train_new.at[row[0], 'covid_afecta'] = 'Yes'
    else:
        X_train_new.at[row[0], 'covid_afecta'] = 'No'

for row in X_test_new.iterrows():
    # Si la moda de les respostes de les variables de la llista 'covid' és 'Yes', la persona ha estat afectada per la covid
    if row[1][covid].mode()[0] == 'Yes':
        X_test_new.at[row[0], 'covid_afecta'] = 'Yes'
    else:
        X_test_new.at[row[0], 'covid_afecta'] = 'No'

# Eliminem les variables de la llista 'covid' ja que ja no les necessitem
X_train_new = X_train_new.drop(covid, axis=1)
X_test_new = X_test_new.drop(covid, axis=1)

print(X_train_new['covid_afecta'].value_counts())

In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train_new.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train_new, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

## Recodificació de variables

In [ ]:
to_cat = ['ID_Zenodo', 'precip_12h_binary', 'precip_24h_binary',
               'noise_total_LDEN_55']

for name in to_cat:
    X_train_new[name] = X_train_new[name].astype('category')
    X_test_new[name] = X_test_new[name].astype('category')

In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train_new.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train_new, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

# Reemplaza 'nan' con np.nan
X_train_new.replace('nan', np.nan, inplace=True)
X_test_new.replace('nan', np.nan, inplace=True)

#contar los valores nulos



In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train_new.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train_new, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Suponiendo que tienes un DataFrame X_train_new que contiene tus datos
# cat_cols contiene las columnas categóricas, excluyendo la columna 'ID_Zenodo'
cat_cols = X_train_new.select_dtypes(include=['object', 'category']).columns
cat_cols = cat_cols.drop('ID_Zenodo')

# Subconjunto de DataFrame con solo las columnas categóricas
X_train_cat = X_train_new[cat_cols]

# Inicializa el codificador OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Ajusta y transforma las columnas categóricas utilizando la codificación one-hot
one_hot_encoded = encoder.fit_transform(X_train_cat)

# Convierte la matriz de codificación one-hot en un DataFrame de pandas
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names(cat_cols))

# Combina el DataFrame codificado one-hot con el resto de tus datos
X_train_encoded = pd.concat([X_train_new.drop(cat_cols, axis=1), one_hot_df], axis=1)

# Ahora X_train_encoded contiene todas las variables originales, pero las variables categóricas han sido codificadas con one-hot


In [ ]:
# Gráfics de barres per variables categòriques
cat_cols = X_train_encoded.select_dtypes(include=['object', 'category']).columns
cat_plots = len(cat_cols)
cat_rows = math.ceil(cat_plots / 3)

fig, axs = plt.subplots(cat_rows, 3, figsize=(15, cat_rows*5))
axs = axs.ravel() 

for i in range(len(axs)):
    if i < len(cat_cols):
        sns.countplot(data=X_train_encoded, x=cat_cols[i], ax=axs[i])
        axs[i].set_title(f'Distribució de {cat_cols[i]}')
        axs[i].set_xlabel(cat_cols[i])
        axs[i].set_ylabel('Comptador')
        axs[i].tick_params(axis='x', rotation=45)
    else:
        fig.delaxes(axs[i])  

plt.tight_layout()
plt.show()

In [ ]:
# Gráfics de distribució per variables numèriques
num_cols = X_train_encoded.select_dtypes(include='number').columns
num_plots = len(num_cols)
num_rows = math.ceil(num_plots / 3)

fig, axs = plt.subplots(num_rows, 3, figsize=(15, num_rows*5))
axs = axs.ravel()  

for i in range(len(axs)):
    if i < len(num_cols):
        sns.histplot(X_train_encoded[num_cols[i]], kde=True, ax=axs[i])
        axs[i].set_title(f'Distribució de {num_cols[i]}')
        axs[i].set_xlabel(num_cols[i])
        axs[i].set_ylabel('Frequència')
    else:
        fig.delaxes(axs[i]) 

plt.tight_layout()
plt.show()


Escalem les variables

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Creem una còpia del DataFrame
X_train_std = X_train_encoded.copy()

# Apliquem la transformació MinMaxScaler a les variables numèriques originals
num_cols = X_train_encoded.select_dtypes(include='number').columns

# Creem un objecte MinMaxScaler i l'ajustem al conjunt de train
scaler = MinMaxScaler()
X_train_std[num_cols] = scaler.fit_transform(X_train_encoded[num_cols])

# Apliquem la mateixa transformació al conjunt de test
X_test_std = X_test_encoded.copy()
X_test_std[num_cols] = scaler.transform(X_test_encoded[num_cols])

# Printejem les estadístiques de les variables numèriques (mínim, màxim i mitjana només) per a train i test
print("Estadístiques de les variables numèriques per a train:")
print(X_train_std[num_cols].describe().loc[['min', 'max', 'mean']])
print("\n")

print("Estadístiques de les variables numèriques per a test:")
print(X_test_std[num_cols].describe().loc[['min', 'max', 'mean']])
print("\n")

## Model de regressió lineal

In [ ]:
# mirem nas a xtrainstd
print(X_train_encoded.isnull().sum())

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

lr = LinearRegression().fit(X_train_std ,y_train)
lr_r2_train = r2_score(y_train, lr.predict(X_train))
lr_r2_val = cross_val_score(lr, X_train, y_train, cv=5, scoring='r2').mean()
lr_r2_test = r2_score(y_test, lr.predict(X_test_std))

In [ ]:
import matplotlib.pyplot as plt
from yellowbrick.regressor import prediction_error

plt.figure(figsize=(8,8))
visualizer = prediction_error(lr, X_test_std, y_test, is_fitted=True)